In [ ]:
!nvidia-smi


# **Dependências**

In [ ]:
# Certifique-se de estar no seu ambiente virtual
# python3 -m venv venv
# source venv/bin/activate

!pip install torch
!pip install xformers==0.0.27
!pip install scikit-learn
!pip install datasets
!pip install trl
!pip install pandas
!pip install numpy
!pip install transformers
!pip install matplotlib
!pip install huggingface_hub
!pip install bitsandbytes
!pip install tyro
!pip install sentencepiece
!pip install tqdm
!pip install psutil
!pip install wheel
!pip install protobuf
!pip install hf_transfer
!pip install tf-keras

# Esta linha é a mais importante para o unsloth
!pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


# **Importação**

In [ ]:
import torch

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Verificar se há GPUs disponíveis
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"GPUs disponíveis: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        # print(f"Memória disponível: {torch.cuda.mem_get_info(i)[0] / 1e9:.2f} GB")
else:
    print("Nenhuma GPU disponível.")


## Importante: Após instalar as dependências, reinicie seu ambiente virtual

# **Início**

In [ ]:
from unsloth import FastLanguageModel

MAX_SEQ_LENGTH = 3500
dtype = None
load_in_4bit = True

MODELOS = ["unsloth/llama-3-8b-bnb-4bit", "unsloth/Qwen2-7B-bnb-4bit","unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit"]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODELOS[0],
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="cuda:0",
)

# **Few-shot**

In [ ]:
from datasets import load_dataset

alpaca_prompt = """Abaixo está uma instrução que descreve uma tarefa, acompanhada de um contexto adicional. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{}

### Contexto:
{}

### Resposta:
{}"""

EOS_TOKEN = tokenizer.eos_token if tokenizer.eos_token else "<|endoftext|>"

def formatting_prompts_func(examples):
    texts = []
    for message in examples['messages']:
        
        instruction = "Você é um especialista em avaliar similaridade de marcas."
        input_text = message[0]['prompt']
        output_text = message[0]['bot_response']

        text = alpaca_prompt.format(instruction, input_text, output_text) + EOS_TOKEN
        texts.append(text)

    return { "text": texts }

dataset = load_dataset("json", data_files="base_de_dados/instruction.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

train_test_split = dataset.train_test_split(test_size=0.1, seed=42)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

print(f"Tamanho do conjunto de treino: {len(train_dataset)}")
print(f"Tamanho do conjunto de teste: {len(test_dataset)}")


In [ ]:
test_dataset[2]

In [ ]:
# IGUAIS
context_prompt = """Você é um especialista em avaliar similaridade de marcas. Em seguida há alguns exemplos de como você deve realizar a análise de similaridade de marcas.
Por exemplo:
- Comando 1: {comando_1}
- Resposta 1: {resposta_1}

- Comando 2: {comando_2}    
- Resposta 2: {resposta_2}
"""

prompt = """
### Instrução:
Agora, você deve avaliar a similaridade entre as seguintes marcas, considerando o contexto fornecido.

### Contexto:
{contexto}

### Resposta:
{resposta}
"""

EOS_TOKEN = tokenizer.eos_token if tokenizer.eos_token else "<|endoftext|>"

context_prompt_formatted = context_prompt.format(
    comando_1 = train_dataset[0]['messages'][0]['prompt'],
    resposta_1 = train_dataset[0]['messages'][0]['bot_response'],
    comando_2 = train_dataset[1]['messages'][0]['prompt'],
    resposta_2 = train_dataset[1]['messages'][0]['bot_response']
)

# depois, train:
indice = 5
input_text = train_dataset[indice]['messages'][0]['prompt']
expected_output = train_dataset[indice]['messages'][0]['bot_response']

query_prompt = prompt.format(contexto=input_text, resposta="")

few_shot_prompt = context_prompt_formatted + "\n\n" + query_prompt + EOS_TOKEN

print(few_shot_prompt)

In [ ]:
# DIFERENTES

context_prompt = """Você é um especialista em avaliar similaridade de marcas. Em seguida há alguns exemplos de como você deve realizar a análise de similaridade de marcas.
Por exemplo:
- Comando 1: {comando_1}
- Resposta 1: {resposta_1}

- Comando 2: {comando_2}    
- Resposta 2: {resposta_2}
"""

prompt = """
### Instrução:
Agora, você deve avaliar a similaridade entre as seguintes marcas, considerando o contexto fornecido.

### Contexto:
{contexto}

### Resposta:
{resposta}
"""

EOS_TOKEN = tokenizer.eos_token if tokenizer.eos_token else "<|endoftext|>"

comando_1 = """Realize uma análise da similaridade entre as marcas nominativas 'LAR PRÁTICO' (Classe Nice: NCL(11) e 'ALTO VALE DISTRIBUIDORA' (Classe Nice: NCL(11)) e explique porque essas marcas são diferentes e não causam risco de confusão ou associação indevida entre os nomes. Considere os seguintes aspectos:\n\n1. Aspecto fonético: Avalie a pronúncia dos nomes levando em conta a entonação, ritmo e sequência das sílabas.\n\n2. Aspecto ideológico: Verifique se os nomes das marcas evocam ideias semelhantes, levando em consideração a classe Nice associada, mesmo que suas pronúncias sejam diferentes.\n\n3. Distintividade: Determine se os nomes possuem elementos dominantes semelhantes.\n\n4. Afinidade mercadológica: Analise a proximidade entre os produtos ou serviços associados às marcas, com base nas classes Nice correspondentes."""
resposta_1 = """
**Análise de diferença entre as marcas nominativas 'LAR PRÁTICO' e 'ALTO VALE DISTRIBUIDORA' (Classe Nice NCL(11))**

1. **Aspecto fonético**:  
   - **Pronúncia**: As marcas "LAR PRÁTICO" e "ALTO VALE DISTRIBUIDORA" possuem ritmos, entonação e sequência de sílabas completamente diferentes. "LAR PRÁTICO" tem uma pronúncia curta e direta, com duas palavras e um total de quatro sílabas. Já "ALTO VALE DISTRIBUIDORA" tem um nome mais longo, com sete sílabas, uma estrutura mais complexa e uma cadência diferente. Além disso, não há sons fonéticos idênticos entre os termos principais de cada marca, o que reduz significativamente qualquer risco de confusão auditiva.

2. **Aspecto ideológico**:  
   - **Ideia evocada**: "LAR PRÁTICO" sugere praticidade e soluções voltadas para o lar, indicando um foco em produtos ou serviços relacionados a organização, manutenção ou itens domésticos. Por outro lado, "ALTO VALE DISTRIBUIDORA" remete a uma localização geográfica específica e a um serviço de distribuição, sugerindo uma empresa que atua no fornecimento e logística de produtos. Como as ideias centrais das marcas são distintas, não há risco de os consumidores associarem um conceito ao outro.

3. **Distintividade**:  
   - **Elementos dominantes**: As palavras-chave principais de cada marca são completamente diferentes. "LAR PRÁTICO" tem como elementos centrais "LAR" e "PRÁTICO", enquanto "ALTO VALE DISTRIBUIDORA" tem como elementos dominantes "ALTO VALE" e "DISTRIBUIDORA". Não há sobreposição significativa de termos nem palavras fonética ou semanticamente similares, o que garante uma alta distintividade entre as duas marcas.

4. **Afinidade mercadológica**:  
   - **Classe Nice 11**: Embora ambas as marcas estejam na mesma classe Nice (11), que pode abranger produtos similares, suas propostas de valor e escopos de atuação são distintos. "LAR PRÁTICO" sugere um foco em itens para o lar, enquanto "ALTO VALE DISTRIBUIDORA" indica uma empresa de distribuição, que pode atuar fornecendo diversos produtos para diferentes setores. Como não há um direcionamento mercadológico evidente que sugira que ambas operam exatamente no mesmo nicho, a possibilidade de confusão entre os consumidores é reduzida.

### **Conclusão**:  
As marcas "LAR PRÁTICO" e "ALTO VALE DISTRIBUIDORA" são suficientemente distintas para evitar qualquer risco de confusão ou associação indevida. Elas diferem significativamente em termos de fonética, ideologia e elementos dominantes, e não apresentam uma afinidade mercadológica forte o suficiente para que os consumidores presumam que pertencem a um mesmo grupo ou fornecem produtos ou serviços equivalentes. Portanto, a coexistência dessas marcas no mercado não representa um risco de confusão para os consumidores.
"""

comando_2 = """Realize uma análise da similaridade entre as marcas nominativas 'INOVE LINGERIE' (Classe Nice NCL(11)) e 
'inovent' (Classe Nice NCL(11) para acessórios de regulagem e segurança para aparelhos de gás)  e explique porque 
essas marcas são diferentes e não causam risco de 
confusão ou associação indevida entre os nomes. Considere os seguintes aspectos:

1. Aspecto fonético: Avalie a pronúncia dos nomes levando em conta a entonação, ritmo e sequência das sílabas.

2. Aspecto ideológico: Verifique se os nomes das marcas evocam ideias semelhantes, levando em consideração a classe
Nice associada, mesmo que suas pronúncias sejam diferentes.

3. Distintividade: Determine se os nomes possuem elementos dominantes semelhantes.

4. Afinidade mercadológica: Analise a proximidade entre os produtos ou serviços associados às marcas, com base nas 
classes Nice correspondentes."""
resposta_2 = """### **Análise da diferença entre as marcas 'INOVE LINGERIE' e 'INOVENT' (Classe Nice NCL(11))**  

1. **Aspecto fonético**:  
   - **Pronúncia**: Embora ambas as marcas compartilhem a raiz "INOV", suas pronúncias são bastante distintas.  
     - **"INOVE LINGERIE"** tem uma pronúncia fluida e suave, com quatro sílabas (**I-NO-VE LIN-GE-RIE**), destacando a entonação francesa na palavra "LINGERIE".  
     - **"INOVENT"** possui uma pronúncia mais curta e firme, com três sílabas (**I-NO-VENT**), terminando com um som mais seco devido à consoante "T".  
   - O ritmo e entonação diferentes dificultam a confusão entre os dois nomes em situações orais ou escritas.  

2. **Aspecto ideológico**:  
   - **"INOVE LINGERIE"** remete ao setor de moda íntima, associando-se a conceitos de sofisticação, feminilidade e design elegante.  
   - **"INOVENT"** sugere inovação e tecnologia industrial, uma vez que se refere a acessórios para regulagem e segurança de aparelhos de gás.  
   - Não há sobreposição conceitual entre os dois, pois um foca no **segmento de vestuário/moda**, enquanto o outro está relacionado a **equipamentos de segurança e regulagem de gás**.  

3. **Distintividade**:  
   - O único elemento semelhante entre as marcas é a raiz "INOV", que pode remeter à inovação.  
   - No entanto, o restante das palavras cria uma diferenciação clara:  
     - **"LINGERIE"** indica diretamente o tipo de produto.  
     - **"ENT"** em "INOVENT" sugere um sufixo mais técnico e industrial.  
   - Essa diferença faz com que os consumidores identifiquem facilmente que se trata de segmentos distintos.  

4. **Afinidade mercadológica**:  
   - Apesar de ambas as marcas estarem registradas na **Classe Nice 11**, essa classe abrange **produtos muito diferentes**.  
   - **"INOVE LINGERIE"** está focada em vestuário íntimo, enquanto **"INOVENT"** atende ao mercado de **acessórios para regulagem e segurança de gás**.  
   - O público-alvo e os canais de distribuição são completamente distintos, reduzindo significativamente qualquer possibilidade de associação indevida.  

---

### **Conclusão:**  
As marcas **"INOVE LINGERIE"** e **"INOVENT"** são claramente diferentes e não apresentam risco de confusão ou associação indevida.  
- **Diferem foneticamente**, possuindo entonações e ritmos distintos.  
- **Evocam conceitos diferentes**, uma relacionada à **moda íntima**, e a outra a **acessórios industriais**.  
- **Os elementos dominantes são distintos**, com "LINGERIE" enfatizando vestuário e "VENT" remetendo a tecnologia e inovação técnica.  
- **Não possuem afinidade mercadológica**, pois atuam em nichos completamente separados dentro da Classe Nice 11.  

Portanto, **não há risco de os consumidores associarem as duas marcas como sendo da mesma empresa ou segmento**."""


context_prompt_formatted = context_prompt.format(
    comando_1 = comando_1,
    resposta_1 = resposta_1,
    comando_2 = comando_2,
    resposta_2 = resposta_2
)

# COMEÇAR PELO 4
indice = 4

contexto = """
Realize uma análise da similaridade entre as marcas nominativas 'ROSSET LINGERIE' (Classe Nice: NCL(8) 35, para agência de importação e exportação) 
e 'INOVE LINGERIE' (Classe Nice: NCL(11) 35, para Comercio através de qualquer meio de lingerie, pijamas, biquíni, camisolas.) e explique porque essas marcas são diferentes e não causam risco de 
confusão ou associação indevida entre os nomes. Considere os seguintes aspectos:

1. Aspecto fonético: Avalie a pronúncia dos nomes levando em conta a entonação, ritmo e sequência das sílabas.

2. Aspecto ideológico: Verifique se os nomes das marcas evocam ideias semelhantes, levando em consideração a classe
Nice associada, mesmo que suas pronúncias sejam diferentes.

3. Distintividade: Determine se os nomes possuem elementos dominantes semelhantes.

4. Afinidade mercadológica: Analise a proximidade entre os produtos ou serviços associados às marcas, com base nas 
classes Nice correspondentes.

Identifique os termos semelhantes, mas explique porque ainda há a diferença entre os nomes. Você deve gerar uma resposta com a mesma estrutura dos exemplos fornecidos.
"""

query_prompt = prompt.format(contexto=contexto, resposta="")

few_shot_prompt = context_prompt_formatted + "\n\n" + query_prompt + EOS_TOKEN

print(few_shot_prompt)

In [ ]:
from rich import print
from rich.text import Text

inputs = tokenizer([few_shot_prompt], return_tensors="pt").to("cuda:0")

outputs = model.generate(**inputs, max_new_tokens = 2500)
resposta = tokenizer.batch_decode(outputs)

print(Text(resposta[0]))
print("Saida esperada: ")
